In [35]:
import pandas as pd

#Tensor flow
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers, Sequential, layers
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [36]:
file_submission = 'data/sample_submission.csv'
file_train_data = 'data/train.csv'
file_test_data = 'data/test.csv'
df_sub = pd.read_csv(file_submission)
df_train = pd.read_csv(file_train_data)
df_test = pd.read_csv(file_test_data)

In [37]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [110]:
# Cut percentage of train vs test
percentage_cut = 20
cut_test = len(df_train) * 20 // 100
cut_train = len(df_train) - cut_test

In [111]:
# Cut percentage of train vs test

X = df_train['text']

tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train)

X_train_token = tokenizer.texts_to_sequences(X_train)


In [115]:
# Padding words
X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='pre')

X_train_pad = X_train_pad[:cut_train]
X_test_pad = X_train_pad[-cut_test:]

y_train = df_train['target'][:cut_train]
y_test = df_train['target'][-cut_test:]

In [116]:
# Initialize model and define vocabulary size
vocab_size = len(tokenizer.word_index)

def initialize_model():
    
    model = Sequential()
    
    model.add(layers.Embedding(
        input_dim= vocab_size + 1,
        input_length= 33, 
        output_dim = 1, 
        mask_zero = True
    ))
    model.add(LSTM(64))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [117]:
model = initialize_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 33, 1)             19323     
                                                                 
 lstm_3 (LSTM)               (None, 64)                16896     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 36284 (141.73 KB)
Trainable params: 36284 (141.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [119]:
es = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(
    X_train_pad, 
    y_train, 
    batch_size=128, 
    epochs=20, 
    validation_data=(X_test_pad, y_test),
    callbacks = [es])

Epoch 1/20
48/48 [==============================] - 9s 182ms/step - loss: 0.6773 - accuracy: 0.5794 - val_loss: 0.6932 - val_accuracy: 0.5342
Epoch 2/20
48/48 [==============================] - 9s 178ms/step - loss: 0.6742 - accuracy: 0.5794 - val_loss: 0.6925 - val_accuracy: 0.5342
Epoch 3/20
48/48 [==============================] - 8s 170ms/step - loss: 0.6673 - accuracy: 0.5794 - val_loss: 0.6936 - val_accuracy: 0.5342
Epoch 4/20
48/48 [==============================] - 9s 179ms/step - loss: 0.6364 - accuracy: 0.6170 - val_loss: 0.7712 - val_accuracy: 0.5440
Epoch 5/20
48/48 [==============================] - 9s 180ms/step - loss: 0.5112 - accuracy: 0.7816 - val_loss: 0.9610 - val_accuracy: 0.5493
Epoch 6/20
48/48 [==============================] - 9s 177ms/step - loss: 0.3593 - accuracy: 0.8539 - val_loss: 1.0752 - val_accuracy: 0.5650
Epoch 7/20
48/48 [==============================] - 8s 173ms/step - loss: 0.2526 - accuracy: 0.9059 - val_loss: 1.2346 - val_accuracy: 0.5598
Epoch 